In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import galsim
from dl import queryClient as qc

In [ ]:
from chromatic_weak_lensing import StellarParams
from chromatic_weak_lensing import utils
from chromatic_weak_lensing.lsst_sim import LSST_Sim
from chromatic_weak_lensing.blackbody import Blackbody
from chromatic_weak_lensing.fsps import FSPS
from chromatic_weak_lensing.btsettl import BTSettl
from chromatic_weak_lensing.phoenix import Phoenix

In [ ]:
filters = {"u", "g", "r", "i", "z", "y"}
throughputs = {
    f: galsim.Bandpass(f"LSST_{f}.dat", "nm").withZeropoint("AB")
    for f in filters
}
red_limit = 12_000

In [ ]:
blackbody = Blackbody()
fsps = FSPS()
btsettl = BTSettl()
phoenix = Phoenix()

In [ ]:
def query_data_lab(ring, n):
    """Fetch TRILEGAL stars for a given ra/dec region."""
    query = \
       """SELECT mass, ra, dec, logg, logte, logl, z, mu0, av, label, c_o, gmag, rmag, imag
          FROM lsst_sim.simdr2
          WHERE (ring256={}) and label=1
          LIMIT {}
       """.format(ring, n)

    print("Submitting request:\n") 
    print(query)

    result = qc.query(sql=query, fmt="structarray")

    return result 

In [ ]:
n_stars = 100
# ring = np.random.choice(range(786432))
ring = 743100
stars = query_data_lab(ring, n_stars)

In [ ]:
lsst_sim = LSST_Sim(stars)

In [ ]:
fig, axs = plt.subplots(
    1, 4,
    sharex=True,
    sharey=True,
    constrained_layout=True,
)

axs[0].set_xlabel("$\lambda$ [$nm$]")
axs[0].set_ylabel("$f_{photons}$ [$photons/nm/cm^2/s$]")
axs[0].set_title("Blackbody")

axs[1].set_xlabel("$\lambda$ [$nm$]")
# axs[1].set_ylabel("$f_{photons}$ [$photons/nm/cm^2/s$]")
axs[1].set_title("FSPS")

axs[2].set_xlabel("$\lambda$ [$nm$]")
# axs[2].set_ylabel("$f_{photons}$ [$photons/nm/cm^2/s$]")
axs[2].set_title("BT-Settl")

axs[3].set_xlabel("$\lambda$ [$nm$]")
# axs[3].set_ylabel("$f_{photons}$ [$photons/nm/cm^2/s$]")
axs[3].set_title("Phoenix")

fig.suptitle("\\textsc{lsst_sim} Stellar Spectra")

wl = np.linspace(300, 1200, 1000)

for i in range(len(lsst_sim.data["mass"])):
    extinction = utils.get_extinction(lsst_sim.data["av"][i], red_limit=red_limit)
    sparams = lsst_sim.get_params(i)
    
    blackbody_params = blackbody.get_params(sparams)
    blackbody_spec = blackbody.get_spectrum(*blackbody_params) * extinction
    
    fsps_params = fsps.get_params(sparams)
    fsps_spec = fsps.get_spectrum(*fsps_params) * extinction

    btsettl_params = btsettl.get_params(sparams)
    btsettl_spec = btsettl.get_spectrum(*btsettl_params) * extinction

    phoenix_params = phoenix.get_params(sparams)
    phoenix_spec = phoenix.get_spectrum(*phoenix_params) * extinction

    axs[0].plot(wl, blackbody_spec(wl))
    axs[1].plot(wl, fsps_spec(wl))
    axs[2].plot(wl, btsettl_spec(wl))
    axs[3].plot(wl, phoenix_spec(wl))

plt.show()

In [ ]:
fig, axs = plt.subplots(
    1, 4,
    sharex=True,
    sharey=True,
    constrained_layout=True,
)

axs[0].set_xlabel("$r$ [\\textsc{lsst_sim}]")
axs[0].set_ylabel("$r$")
axs[0].set_title("Blackbody")

axs[1].set_xlabel("$r$ [\\textsc{lsst_sim}]")
axs[1].set_title("FSPS")

axs[2].set_xlabel("$r$ [\\textsc{lsst_sim}]")
axs[2].set_title("BT-Settl")

axs[3].set_xlabel("$r$ [\\textsc{lsst_sim}]")
axs[3].set_title("Phoenix")

fig.suptitle("\\textsc{lsst_sim} Magnitudes")

lsst_sim_magnitudes = []
blackbody_magnitudes = []
fsps_magnitudes = []
btsettl_magnitudes = []
phoenix_magnitudes = []

for i in range(len(lsst_sim.data["mass"])):
    extinction = utils.get_extinction(lsst_sim.data["av"][i], red_limit=red_limit)
    sparams = lsst_sim.get_params(i)

    lsst_sim_magnitude = lsst_sim.data["rmag"][i]
    lsst_sim_magnitudes.append(lsst_sim_magnitude)

    blackbody_params = blackbody.get_params(sparams)
    blackbody_spec = blackbody.get_spectrum(*blackbody_params) * extinction
    blackbody_magnitude = blackbody_spec.calculateMagnitude(throughputs["r"])
    blackbody_magnitudes.append(blackbody_magnitude)
    
    fsps_params = fsps.get_params(sparams)
    fsps_spec = fsps.get_spectrum(*fsps_params) * extinction
    fsps_magnitude = fsps_spec.calculateMagnitude(throughputs["r"])
    fsps_magnitudes.append(fsps_magnitude)
    
    btsettl_params = btsettl.get_params(sparams)
    btsettl_spec = btsettl.get_spectrum(*btsettl_params) * extinction
    btsettl_magnitude = btsettl_spec.calculateMagnitude(throughputs["r"])
    btsettl_magnitudes.append(btsettl_magnitude)

    phoenix_params = phoenix.get_params(sparams)
    phoenix_spec = phoenix.get_spectrum(*phoenix_params) * extinction
    phoenix_magnitude = phoenix_spec.calculateMagnitude(throughputs["r"])
    phoenix_magnitudes.append(phoenix_magnitude)

axs[0].plot(
    [np.min(lsst_sim_magnitudes), np.max(lsst_sim_magnitudes)],
    [np.min(lsst_sim_magnitudes), np.max(lsst_sim_magnitudes)],
    c="gray",
    ls="--",
)
axs[0].scatter(lsst_sim_magnitudes, blackbody_magnitudes, c="k")

axs[1].plot(
    [np.min(lsst_sim_magnitudes), np.max(lsst_sim_magnitudes)],
    [np.min(lsst_sim_magnitudes), np.max(lsst_sim_magnitudes)],
    c="gray",
    ls="--",
)
axs[1].scatter(lsst_sim_magnitudes, fsps_magnitudes, c="k")

axs[2].plot(
    [np.min(lsst_sim_magnitudes), np.max(lsst_sim_magnitudes)],
    [np.min(lsst_sim_magnitudes), np.max(lsst_sim_magnitudes)],
    c="gray",
    ls="--",
)
axs[2].scatter(lsst_sim_magnitudes, btsettl_magnitudes, c="k")

axs[3].plot(
    [np.min(lsst_sim_magnitudes), np.max(lsst_sim_magnitudes)],
    [np.min(lsst_sim_magnitudes), np.max(lsst_sim_magnitudes)],
    c="gray",
    ls="--",
)
axs[3].scatter(lsst_sim_magnitudes, phoenix_magnitudes, c="k")

plt.show()

In [ ]:
fig, axs = plt.subplots(
    1, 4,
    sharex=True,
    sharey=True,
    constrained_layout=True,
)

axs[0].set_xlabel("$g - i$ [\\textsc{lsst_sim}]")
axs[0].set_ylabel("$g - i$")
axs[0].set_title("Blackbody")

axs[1].set_xlabel("$g - i$ [\\textsc{lsst_sim}]")
axs[1].set_title("FSPS")

axs[2].set_xlabel("$g - i$ [\\textsc{lsst_sim}]")
axs[2].set_title("BT-Settl")

axs[3].set_xlabel("$g - i$ [\\textsc{lsst_sim}]")
axs[3].set_title("Phoenix")

fig.suptitle("\\textsc{lsst_sim} Colors")

lsst_sim_colors = []
blackbody_colors = []
fsps_colors = []
btsettl_colors = []
phoenix_colors = []

for i in range(len(lsst_sim.data["mass"])):
    extinction = utils.get_extinction(lsst_sim.data["av"][i], red_limit=red_limit)
    sparams = lsst_sim.get_params(i)

    lsst_sim_color = lsst_sim.data["gmag"][i] - lsst_sim.data["imag"][i]
    lsst_sim_colors.append(lsst_sim_color)

    blackbody_params = blackbody.get_params(sparams)
    blackbody_spec = blackbody.get_spectrum(*blackbody_params) * extinction
    blackbody_color = blackbody_spec.calculateMagnitude(throughputs["g"]) - blackbody_spec.calculateMagnitude(throughputs["i"])
    blackbody_colors.append(blackbody_color)
    
    fsps_params = fsps.get_params(sparams)
    fsps_spec = fsps.get_spectrum(*fsps_params) * extinction
    fsps_color = fsps_spec.calculateMagnitude(throughputs["g"]) - fsps_spec.calculateMagnitude(throughputs["i"])
    fsps_colors.append(fsps_color)
    
    btsettl_params = btsettl.get_params(sparams)
    btsettl_spec = btsettl.get_spectrum(*btsettl_params) * extinction
    btsettl_color = btsettl_spec.calculateMagnitude(throughputs["g"]) - btsettl_spec.calculateMagnitude(throughputs["i"])
    btsettl_colors.append(btsettl_color)

    phoenix_params = phoenix.get_params(sparams)
    phoenix_spec = phoenix.get_spectrum(*phoenix_params) * extinction
    phoenix_color = phoenix_spec.calculateMagnitude(throughputs["g"]) - phoenix_spec.calculateMagnitude(throughputs["i"])
    phoenix_colors.append(phoenix_color)

axs[0].plot(
    [np.min(lsst_sim_colors), np.max(lsst_sim_colors)],
    [np.min(lsst_sim_colors), np.max(lsst_sim_colors)],
    c="gray",
    ls="--",
)
axs[0].scatter(lsst_sim_colors, blackbody_colors, c="k")

axs[1].plot(
    [np.min(lsst_sim_colors), np.max(lsst_sim_colors)],
    [np.min(lsst_sim_colors), np.max(lsst_sim_colors)],
    c="gray",
    ls="--",
)
axs[1].scatter(lsst_sim_colors, fsps_colors, c="k")

axs[2].plot(
    [np.min(lsst_sim_colors), np.max(lsst_sim_colors)],
    [np.min(lsst_sim_colors), np.max(lsst_sim_colors)],
    c="gray",
    ls="--",
)
axs[2].scatter(lsst_sim_colors, btsettl_colors, c="k")

axs[3].plot(
    [np.min(lsst_sim_colors), np.max(lsst_sim_colors)],
    [np.min(lsst_sim_colors), np.max(lsst_sim_colors)],
    c="gray",
    ls="--",
)
axs[3].scatter(lsst_sim_colors, phoenix_colors, c="k")

plt.show()